# NGC346-MPG-355

Updating the dataset: P203030100000

* The guide channel is LiF 1, the LiF 1A and 1B count rates are rock steady, and the LiF 2B spectrum matches LiF 1A pretty well.  
* Count rates for the SiC channels suggest that the star drifts out of their apertures, yet their fluxes are higher. Normally, we would assume that both of the SiC channels are contaminated, but in this case, we assume that the SiC 1A channel is correct and scale the other channels to match.  
* With this assumption, the FUSE spectrum matches the STIS spectrum in the region of overlap.  
* The flux of LIF 2A falls with increasing wavelength, probably because of the worm, so we use LiF 1B where available.
* Note: A star in the LWRS aperture was included in the background calculation. I have re-defined the background regions and re-extracted all of the spectra.

COMMENT File updated 5 August 2022<br>
COMMENT This NVO file has been reconstructed from the corresponding ALL file.<br>
COMMENT All segments have been scaled to match the SiC 1 channel.<br>
COMMENT Where available, we use LiF 1B rather than LiF 2A.

In [ ]:
%matplotlib inline
import matplotlib

import numpy as np

from astropy import units as u
from astropy.convolution import convolve, Box1DKernel
from astropy.io import fits
from astropy.visualization import quantity_support
from matplotlib import pyplot as plt

from specutils import Spectrum1D
from specutils.spectra import SpectralRegion
from specutils.manipulation import box_smooth, extract_region

# Specify plot parameters.
quantity_support()  # to put units on the axes below 
matplotlib.rcParams['figure.figsize'] = [15, 5]

# Wavelength per channel (pixel) is always 0.013 A.
WPC = 0.013

# Normalize all spectra, so fluxes are of order unity.
norm = 1E12

In [ ]:
# Function to convert FITS array into spectrum object.

def make_spectrum (specdata):
    lamb = specdata['wave'] * u.AA 
    flux = specdata['flux'] * norm * u.Unit('erg cm-2 s-1 AA-1') 
    return Spectrum1D(spectral_axis=lamb, flux=flux)

In [ ]:
# Function to compute offset in pixels between two spectra

def compute_shift(specdata, refdata, wmin, wmax):
    wave_spec  = specdata['wave']
    flux_spec  = specdata['flux']
    error_spec = specdata['error']
    wave_ref   =  refdata['wave']
    flux_ref   =  refdata['flux']
    
    # Compute scale factor between data and reference spectra.
    g = np.where((wave_spec > wmin) & (wave_spec < wmax))
    h = np.where((wave_ref  > wmin) & (wave_ref  < wmax))
    spec_mean = np.mean(flux_spec[g])
    ref_mean  = np.mean(flux_ref[h])
    scale = ref_mean / spec_mean

    # Smooth data and error arrays.
    flux_spec  = convolve(flux_spec,  Box1DKernel(7))
    error_spec = convolve(error_spec, Box1DKernel(7))
    flux_ref   = convolve(flux_ref,   Box1DKernel(7))
    
    # Rescale spectrum to match reference.
    flux_spec *= scale
    error_spec *= scale
    
    # Compute minimum value of chi-squared.
    chisq = np.zeros(20)
    for i in range(20):
        j = i - 10
        chisq[i] = np.sum(((flux_spec[g[0]+j] - flux_ref[h]) / error_spec[g[0]+j])**2)
    
    # We have computed this offset by shifting the flux array, but will 
    # use it by shifting the wavelength array, so must multiply by -1.
    return 10 - np.argmin(chisq)

In [ ]:
# Function to read a regular FUSE spectral file.

def read_fuse(filename):
    f = fits.open(filename)
    data = f[1].data
    f.close()
    spec = make_spectrum (data)
    return data, spec

In [ ]:
# Read header keywords.

filename = 'p203030100000all2ttagfcal.fit'
f = fits.open(filename)
print ('Target:  ', f[0].header['TARGNAME'])
print ('Aperture:', f[0].header['APERTURE'])
print ('Guider:  ', f[0].header['FESCENT'])
print ('CHANL OBSTIME COMBMETH')
for i in range(1,9): print (f[i].header['EXTNAME'], f[i].header['OBSTIME'], f[i].header['COMBMETH'])
    
# The guide channel was LiF 1.

In [ ]:
# Read all eight spectra from FUSE *all* file.

lif1a_data = f['1alif'].data 
lif1b_data = f['1blif'].data
lif2b_data = f['2blif'].data
lif2a_data = f['2alif'].data
sic1a_data = f['1asic'].data
sic1b_data = f['1bsic'].data
sic2b_data = f['2bsic'].data
sic2a_data = f['2asic'].data
f.close() 

In [ ]:
# Convert FITS arrays into spectrum objects.

lif1a = make_spectrum (lif1a_data)
lif1b = make_spectrum (lif1b_data)
lif2a = make_spectrum (lif2a_data)
lif2b = make_spectrum (lif2b_data)
sic1a = make_spectrum (sic1a_data)
sic1b = make_spectrum (sic1b_data)
sic2a = make_spectrum (sic2a_data)
sic2b = make_spectrum (sic2b_data)

In [ ]:
# Smooth the spectral arrays.

lif1a_bsmooth = box_smooth(lif1a, width=15)
lif1b_bsmooth = box_smooth(lif1b, width=15)
lif2a_bsmooth = box_smooth(lif2a, width=15)
lif2b_bsmooth = box_smooth(lif2b, width=15)
sic1a_bsmooth = box_smooth(sic1a, width=15)
sic1b_bsmooth = box_smooth(sic1b, width=15)
sic2a_bsmooth = box_smooth(sic2a, width=15)
sic2b_bsmooth = box_smooth(sic2b, width=15)

In [ ]:
# Both the LiF 2B and SiC 2B channels have background problems.

f, ax = plt.subplots()  
ax.step(sic1a_bsmooth.spectral_axis, sic1a_bsmooth.flux, label='SiC 1A') 
ax.step(sic2b_bsmooth.spectral_axis, sic2b_bsmooth.flux, label='SiC 2B')
ax.step(lif1a_bsmooth.spectral_axis, lif1a_bsmooth.flux, label='LiF 1A') 
ax.step(lif2b_bsmooth.spectral_axis, lif2b_bsmooth.flux, label='LiF 2B')
ax.legend()
ax.set_xlim([980,1050])
ax.set_ylim([-0.5,2])

# Let's fix them before proceeding.

In [ ]:
# For all channels, a star in the 
# LWRS aperture was included in the background calculation. 
# I have re-defined the background regions and re-extracted all of the spectra.
# Let's look at the results.

lif1a_data, lif1a = read_fuse('P20303010001alif2ttagfcal.fit')
lif1b_data, lif1b = read_fuse('P20303010001blif2ttagfcal.fit')
lif2a_data, lif2a = read_fuse('P20303010002alif2ttagfcal.fit')
lif2b_data, lif2b = read_fuse('P20303010002blif2ttagfcal.fit')
sic1a_data, sic1a = read_fuse('P20303010001asic2ttagfcal.fit')
sic1b_data, sic1b = read_fuse('P20303010001bsic2ttagfcal.fit')
sic2a_data, sic2a = read_fuse('P20303010002asic2ttagfcal.fit')
sic2b_data, sic2b = read_fuse('P20303010002bsic2ttagfcal.fit')

In [ ]:
# Smooth the spectral arrays.

lif1a_bsmooth = box_smooth(lif1a, width=15)
lif1b_bsmooth = box_smooth(lif1b, width=15)
lif2a_bsmooth = box_smooth(lif2a, width=15)
lif2b_bsmooth = box_smooth(lif2b, width=15)
sic1a_bsmooth = box_smooth(sic1a, width=15)
sic1b_bsmooth = box_smooth(sic1b, width=15)
sic2a_bsmooth = box_smooth(sic2a, width=15)
sic2b_bsmooth = box_smooth(sic2b, width=15)

In [ ]:
# Plot the smoothed combined spectra.  

f, ax = plt.subplots()  
ax.step(sic1a_bsmooth.spectral_axis, sic1a_bsmooth.flux, label='SiC 1A') 
ax.step(sic2b_bsmooth.spectral_axis, sic2b_bsmooth.flux, label='SiC 2B')
ax.step(lif1a_bsmooth.spectral_axis, lif1a_bsmooth.flux, label='LiF 1A') 
ax.step(lif2b_bsmooth.spectral_axis, lif2b_bsmooth.flux, label='LiF 2B')
ax.legend()
ax.set_xlim([980,1050])
ax.set_ylim([-0.5,2])

# This looks better.

In [ ]:
# Plot the smoothed combined spectra.  
# Note that spectra are scaled to have values near unity.

f, ax = plt.subplots()  
ax.step(sic1a_bsmooth.spectral_axis, sic1a_bsmooth.flux, label='SiC 1A') 
ax.step(sic2b_bsmooth.spectral_axis, sic2b_bsmooth.flux, label='SiC 2B')
ax.step(lif1a_bsmooth.spectral_axis, lif1a_bsmooth.flux, label='LiF 1A') 
ax.step(lif2b_bsmooth.spectral_axis, lif2b_bsmooth.flux, label='LiF 2B')
ax.legend()
ax.set_xlim([1000,1100])
ax.set_ylim([-0.5,2])

# The guide channel is LiF 1, the LiF 1A and 1B count rates are rock steady, and the
# LiF 2B spectrum matches LiF 1A pretty well.  Count rates for the SiC channels suggest
# that the star drifts out of their apertures, yet their fluxes are higher.  
# Normally, we would assume that both SiC channels are contaminated, but in this case,
# we will assume that the SiC 1A channel is correct and scale the other channels to match.

In [ ]:
# Select a spectral region.

region = SpectralRegion(1050*u.AA, 1070*u.AA)
sub_lif1a = extract_region(lif1a, region)
sub_lif2b = extract_region(lif2b, region)
sub_sic1a = extract_region(sic1a, region)
sub_sic2b = extract_region(sic2b, region)

# Compute ratio of their fluxes.

scale_lif1a= sub_sic1a.mean()/sub_lif1a.mean()
print ('Scale LiF 1A by', scale_lif1a)

scale_lif2b = sub_sic1a.mean()/sub_lif2b.mean()
print ('Scale LiF 2B by', scale_lif2b)

scale_sic2b = sub_sic1a.mean()/sub_sic2b.mean()
print ('Scale SiC 2B by', scale_sic2b)

In [ ]:
# Scale the spectra.

lif1a *= scale_lif1a

lif1a_bsmooth *= scale_lif1a
lif2b_bsmooth *= scale_lif2b
sic2b_bsmooth *= scale_sic2b

In [ ]:
# Plot the rescaled spectra.

f, ax = plt.subplots()  
ax.step(sic1a_bsmooth.spectral_axis, sic1a_bsmooth.flux, label='SiC 1A') 
ax.step(sic2b_bsmooth.spectral_axis, sic2b_bsmooth.flux, label='SiC 2B')
ax.step(lif1a_bsmooth.spectral_axis, lif1a_bsmooth.flux, label='LiF 1A') 
ax.step(lif2b_bsmooth.spectral_axis, lif2b_bsmooth.flux, label='LiF 2B')
ax.legend()
ax.set_xlim([1000,1100])
ax.set_ylim([-0.5,2])

# The spectra match pretty well.  We assume that they represent the same star.

In [ ]:
# Which channel has the hightest S/N?

f, ax = plt.subplots()  
ax.step(lif1a.spectral_axis, float(scale_lif1a) * lif1a_data['error'], label='LiF1A') 
ax.step(lif2b.spectral_axis, float(scale_lif2b) * lif2b_data['error'], label='LiF2B') 
ax.step(sic1a.spectral_axis, sic1a_data['error'], label='SiC1A')
ax.step(sic2b.spectral_axis, float(scale_sic2b) * sic2b_data['error'], label='SiC2B')
ax.legend()
ax.set_xlim([1040,1080])
ax.set_ylim([0, 4E-13])

# So LiF 1A still yields the highest S/N.

In [ ]:
# At longer wavelengths, the LiF 2A flux falls with increasing wavelength.
# Image plots reveal a worm between about 1120 and 1180 A that might explain this effect.
# Still, it is odd the LiF 1A and LiF 2B spectra agree, while the LiF 1B and LiF 2A do not.

f, ax = plt.subplots()  
ax.step(lif1b_bsmooth.spectral_axis, lif1b_bsmooth.flux, label='LiF 1B') 
ax.step(lif2a_bsmooth.spectral_axis, lif2a_bsmooth.flux, label='LiF 2A') 
ax.legend()

In [ ]:
# LiF 1A and LiF 1B have nearly equal exposure times.  Let's see if the scale factor for LiF 1A works
# for LiF 1B.

lif1b *= scale_lif1a

lif1b_bsmooth *= scale_lif1a

In [ ]:
# Compare (rescaled) LiF 1B with a STIS spectrum of the same star.

stis = Spectrum1D.read("oe9e2s010_x1d.fits", format="HST/STIS")
print (fits.getval("oe9e2s010_x1d.fits", 'TARGNAME', 0))
print (fits.getval("oe9e2s010_x1d.fits", 'APERTURE', 0))
stis_bsmooth = box_smooth(stis, width=15)

# According to Simbad, NGC 346 W 3 and NGC 346 MPG 355 are the same star.

f, ax = plt.subplots()
ax.plot(stis_bsmooth.spectral_axis, norm * stis_bsmooth.flux, label='STIS')  
ax.plot(lif1b_bsmooth.spectral_axis, lif1b_bsmooth.flux, label='LiF 1B') 
ax.legend()
ax.set_xlim([1140,1200])
ax.set_ylim([0,2])

# This looks pretty good.  We'll continue with the assumption that SiC 1A yields the correct flux.

In [ ]:
# If we are happy with LiF 1B, we should scale LiF 2A to match it near 1100 A.

# Select a spectral region.

region = SpectralRegion(1100*u.AA, 1120*u.AA)
sub_lif1b = extract_region(lif1b, region)
sub_lif2a = extract_region(lif2a, region)

# Compute ratio of their fluxes.

scale_lif2a = sub_lif1b.mean()/sub_lif2a.mean()
print ('Scale LiF 2A by', scale_lif2a)

In [ ]:
# Scale the spectrum.

lif2a_bsmooth *= scale_lif2a

In [ ]:
# Plot the rescaled spectrum.

f, ax = plt.subplots()  
ax.step(lif1b_bsmooth.spectral_axis, lif1b_bsmooth.flux, label='LiF 1B') 
ax.step(lif2a_bsmooth.spectral_axis, lif2a_bsmooth.flux, label='LiF 2A') 
ax.legend()
# ax.set_xlim([1090,1120])

In [ ]:
# At short wavelengths, we assume that SiC 1B is correct and scale SiC 2A to match.

# Select a spectral region.

region = SpectralRegion(957*u.AA, 970*u.AA)
sub_sic1b = extract_region(sic1b, region)
sub_sic2a = extract_region(sic2a, region)

# Compute ratio of the fluxes.

scale_sic2a = sub_sic1b.mean()/sub_sic2a.mean()
print ('Scale SiC 2A by', scale_sic2a)

# Scale the spectrum.

sic2a_bsmooth *= scale_sic2a

# Plot the scaled spectra.

f, ax = plt.subplots() 
ax.step(lif1a_bsmooth.spectral_axis, lif1a_bsmooth.flux, label='LiF 1A')
ax.step(sic1b_bsmooth.spectral_axis, sic1b_bsmooth.flux, label='SiC 1B')
ax.step(sic2a_bsmooth.spectral_axis, sic2a_bsmooth.flux, label='SiC 2A')
ax.legend()
ax.set_xlim([980, 1010])
ax.set_ylim([0,2])

In [ ]:
# Next we compute the shift (in pixels) relative to LiF 1A (which still has the highest S/N).

shift_lif2 = compute_shift(lif2b_data, lif1a_data, 1040, 1070)
shift_sic1 = compute_shift(sic1a_data, lif1a_data, 1040, 1070)
shift_sic2 = compute_shift(sic2b_data, lif1a_data, 1040, 1070)

print ('Shift LiF 2 by ', shift_lif2, ' pixels.')
print ('Shift SiC 1 by ', shift_sic1, ' pixels.')
print ('Shift SiC 2 by ', shift_sic2, ' pixels.')

In [ ]:
# Shift all eight spectra.

shift_lif2 *= WPC
shift_sic1 *= WPC
shift_sic2 *= WPC

lif1a_shift = lif1a
lif1b_shift = lif1b
lif2a_shift = Spectrum1D(spectral_axis=lif2a.spectral_axis + shift_lif2 * u.AA, flux=lif2a.flux)
lif2b_shift = Spectrum1D(spectral_axis=lif2b.spectral_axis + shift_lif2 * u.AA, flux=lif2b.flux)
sic1a_shift = Spectrum1D(spectral_axis=sic1a.spectral_axis + shift_sic1 * u.AA, flux=sic1a.flux)
sic1b_shift = Spectrum1D(spectral_axis=sic1b.spectral_axis + shift_sic1 * u.AA, flux=sic1b.flux)
sic2a_shift = Spectrum1D(spectral_axis=sic2a.spectral_axis + shift_sic2 * u.AA, flux=sic2a.flux)
sic2b_shift = Spectrum1D(spectral_axis=sic2b.spectral_axis + shift_sic2 * u.AA, flux=sic2b.flux)

f, ax = plt.subplots()  
ax.step(lif1a.spectral_axis, lif1a_bsmooth.flux, label='LiF 1A') 
ax.step(lif2b_shift.spectral_axis, lif2b_bsmooth.flux, label='LiF 2B')
ax.step(sic1a_shift.spectral_axis, sic1a_bsmooth.flux, label='SiC 1A') 
ax.step(sic2b_shift.spectral_axis, sic2b_bsmooth.flux, label='SiC 2B')
ax.legend()
ax.set_xlim([1060,1070])
ax.set_ylim([0,2])

In [ ]:
# We reconstruct the spectrum from the ALL file.  We do not rescale the SiC 1 channels.
# Because of the worm, we use LiF 1B instead of LiF 2A at long wavelengths.

filename = 'p203030100000nvo2ttagfcal.fit'
f = fits.open(filename)
hdr = f[0].header
data = f[1].data 

# Scale SiC 1B
sic1b_wave = sic1b_shift.spectral_axis.value
h = np.where((sic1b_wave > 899.99) & (sic1b_wave < 917.5))
g = np.where((data['wave'] > np.min(sic1b_wave[h]) - 0.01) & (data['wave'] < 917.5))
data['flux'][g] = sic1b_data['flux'][h]
data['error'][g] = sic1b_data['error'][h]

# Scale SiC 2A
sic2a_wave = sic2a_shift.spectral_axis.value
g = np.where((data['wave'] > 917.5) & (data['wave'] < 998))
h = np.where((sic2a_wave > 917.5) & (sic2a_wave < 998))
data['flux'][g] = float(scale_sic2a) * sic2a_data['flux'][h]
data['error'][g] = float(scale_sic2a) * sic2a_data['error'][h]

# Scale LiF 1A
lif1a_wave = lif1a_shift.spectral_axis.value
g = np.where((data['wave'] > 998) & (data['wave'] < 1082.5))
h = np.where((lif1a_wave > 998) & (lif1a_wave < 1082.5))
data['flux'][g] = float(scale_lif1a) * lif1a_data['flux'][h]
data['error'][g] = float(scale_lif1a) * lif1a_data['error'][h]

# Scale SiC 1A 
sic1a_wave = sic1a_shift.spectral_axis.value
g = np.where((data['wave'] > 1082.5) & (data['wave'] < 1087))
h = np.where((sic1a_wave > 1082.5) & (sic1a_wave < 1087))
data['flux'][g] = sic1a_data['flux'][h]
data['error'][g] = sic1a_data['error'][h]

# Scale LiF 2A
lif2a_wave = lif2a_shift.spectral_axis.value
g = np.where((data['wave'] > 1087) & (data['wave'] < 1094))
h = np.where((lif2a_wave > 1087) & (lif2a_wave < 1094))
data['flux'][g] = float(scale_lif2a) * lif2a_data['flux'][h]
data['error'][g] = float(scale_lif2a) * lif2a_data['error'][h]

# Scale LiF 1B (using the scale factor for LiF 1A)
lif1b_wave = lif1b_shift.spectral_axis.value
h = np.where((lif1b_wave > 1094) & (lif1b_wave < 1190.01))
g = np.where((data['wave'] > 1094) & (data['wave'] < np.max(lif1b_wave[h]) + 0.01))
data['flux'][g] = float(scale_lif1a) * lif1b_data['flux'][h]
data['error'][g] = float(scale_lif1a) * lif1b_data['error'][h]

hdr['comment'] = ''
hdr['comment'] = 'File updated 5 August 2022'
hdr['comment'] = 'This NVO file has been reconstructed from the corresponding ALL file.'
hdr['comment'] = 'All segments have been scaled to match the SiC 1 channel.'
hdr['comment'] = 'Where available, we use LiF 1B rather than LiF 2A.'

f.writeto('level0_p203030100000nvo2ttagfcal_vo.fit', overwrite=True)
f.close()

In [ ]:
# Compare old and new versions of NVO file.

filename = 'p203030100000nvo2ttagfcal.fit'
f = fits.open(filename)
old = f[1].data 
f.close()

filename = 'level0_p203030100000nvo2ttagfcal_vo.fit'
f = fits.open(filename)
hdr = f[0].header
new = f[1].data 
f.close()

print (hdr['comment'])

f, (ax1, ax2) = plt.subplots(2, 1, sharey=True)  
ax1.step(old['wave'], old['flux'], label='OLD FLUX') 
ax1.step(new['wave'], new['flux'], label='NEW FLUX')
ax2.step(old['wave'], old['flux'], label='OLD FLUX') 
ax2.step(new['wave'], new['flux'], label='NEW FLUX')
ax1.legend()
ax1.set_xlim([900, 1050])
ax2.set_xlim([1050, 1190])
ax1.set_ylim([0,3E-12])